In [27]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import re

In [28]:
url = 'https://program.naver.com/p/36219184/community/rating'
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

view_more = driver.find_element(By.XPATH, '//*[@id="cbox_module"]/div/div[7]/a/span/span/span[1]')

while True:
    try:
        view_more.click()
        time.sleep(5)
    except Exception as e:
        break

Ratings = []
Minus_Dates = []
Reviews = []

for i in range(1, 126):
    rating =  driver.find_element(By.XPATH, '//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[{}]/div[1]/div/div[1]/span[2]'.format(1*i)).text
    ago = driver.find_element(By.XPATH, '//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[{}]/div[1]/div/div[4]/span[1]'.format(1*i)).text
    review = driver.find_element(By.XPATH, '//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[{}]/div[1]/div/div[3]/span'.format(1*i)).text

    Ratings.append(rating)
    Minus_Dates.append(ago)
    Reviews.append(review)

In [29]:
# Minus_Dates
print(len(Ratings), len(Minus_Dates), len(Reviews))

125 125 125


In [30]:
typhoon_family_raw = {'Ratings': Ratings, 
                           'Minus_Dates': Minus_Dates,
                           'Reviews': Reviews}

typhoon_family_df = pd.DataFrame(typhoon_family_raw)

In [31]:
typhoon_family_df

,Ratings,Minus_Dates,Reviews
0,10,5시간 전,여기가 엔딩맛집이라 하는 태풍상사인가요?\n토일 드라마라서 평일내내 사람을 괴롭게 ...
1,10,1일 전,"고난의 연속이 성장드리미지만, 간긴히 기분 좋은 한빙으로 한 계단씩 성장하믄 따숩고..."
2,5,1일 전,태풍엄마!! 너무 오글거려 못보겄어 ㅡㅡ그게 대본이고 연출이라면 어쩔수 없겠으나 보...
3,10,2일 전,마음이 따뜻해지는 드라마. 너무 재밌어요.
4,10,2일 전,오랜만에 아주 좋은 드라마 만났네요
...,...,...,...
120,10,2025.10.12. 16:00,진짜 보고 또 보고 싶은 드라마 <태풍상사>\n이준호 연기 ㄹㅈㄷ\n예능 나와서 홍...
121,4,2025.10.12. 11:22,고증 잘 햇다는 글은 그 때 태어나기전 사람들이거나 알바인가? 예전은 다 촌스럽다는...
122,10,2025.10.12. 08:03,"1화, 시대고증, 캐릭터, 연기, 연출, 음악, 미술 다 좋았음. 특히 엔딩 연기 ..."
123,10,2025.10.11. 23:27,"1화. 초반엔 SNL, 응답하라1997인듯 재미있고, 중반에는 미생처럼 상사 이야기..."


In [32]:
hr_pattern = r'시간 전'
day_pattern = r'일 전'

def time_conversion(text):
    if pd.isna(text):
        return pd.NaT
    
    # Hours ago
    hr_match = re.search(r'(\d+)\s*시간 전', text)
    if hr_match:
        hours = int(hr_match.group(1))
        return pd.Timestamp.now() - pd.to_timedelta(hours, unit='h')
    
    # Days pattern
    day_match = re.search(r'(\d+)\s*일 전', text)
    if day_match:
        days = int(day_match.group(1))
        return pd.Timestamp.now() - pd.to_timedelta(days, unit='d')
    
    # If no known pattern
    date_match = re.search(r'\d{4}\.\d{2}\.\d{2}\.\s*\d{2}:\d{2}', text)
    if date_match:
        try:
            return pd.to_datetime(text.strip(), format='%Y.%m.%d. %H:%M', errors='coerce')
        except Exception:
            return pd.NaT

    # Default fallback
    return pd.NaT
        
typhoon_family_df['Review_Dates'] = typhoon_family_df['Minus_Dates'].apply(time_conversion)


In [33]:
typhoon_family_df['Review_Dates'] = typhoon_family_df['Review_Dates'].dt.date

In [34]:
typhoon_family_df_final = typhoon_family_df.drop('Minus_Dates', axis=1)

In [35]:
typhoon_family_df_final

,Ratings,Reviews,Review_Dates
0,10,여기가 엔딩맛집이라 하는 태풍상사인가요?\n토일 드라마라서 평일내내 사람을 괴롭게 ...,2025-10-22
1,10,"고난의 연속이 성장드리미지만, 간긴히 기분 좋은 한빙으로 한 계단씩 성장하믄 따숩고...",2025-10-21
2,5,태풍엄마!! 너무 오글거려 못보겄어 ㅡㅡ그게 대본이고 연출이라면 어쩔수 없겠으나 보...,2025-10-21
3,10,마음이 따뜻해지는 드라마. 너무 재밌어요.,2025-10-20
4,10,오랜만에 아주 좋은 드라마 만났네요,2025-10-20
...,...,...,...
120,10,진짜 보고 또 보고 싶은 드라마 <태풍상사>\n이준호 연기 ㄹㅈㄷ\n예능 나와서 홍...,2025-10-12
121,4,고증 잘 햇다는 글은 그 때 태어나기전 사람들이거나 알바인가? 예전은 다 촌스럽다는...,2025-10-12
122,10,"1화, 시대고증, 캐릭터, 연기, 연출, 음악, 미술 다 좋았음. 특히 엔딩 연기 ...",2025-10-12
123,10,"1화. 초반엔 SNL, 응답하라1997인듯 재미있고, 중반에는 미생처럼 상사 이야기...",2025-10-11


In [37]:
typhoon_family_df_final.to_csv('typhoon_family.csv')